In [101]:
import numpy as np
import matplotlib.pyplot as plt
import random
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm

# Build model

In [21]:
input_dim=2
hidden_dim = 512
output_dim = 2

In [6]:
torch.ones(512, requires_grad=False).dtype

torch.float32

In [51]:
class DropoutLayer(nn.Module):
    def __init__(self, hidden_dim=hidden_dim):
        super(DropoutLayer, self).__init__()
        self.mask = torch.ones(hidden_dim, dtype=torch.float32, requires_grad=False)
        self.training = False
    def forward(self, x):
        return x * self.mask
    def update(self, mask: np.ndarray):
        assert mask.shape == self.mask.shape, f"new mask shape should be {self.mask.shape} but giving {mask.shape}"
        self.mask = torch.Tensor(mask)
    def get(self):
        return self.mask.numpy()

In [52]:
dropout = DropoutLayer()

In [53]:
dropout.get().shape

(512,)

In [54]:
a = torch.rand(512)

In [55]:
b = dropout(a)

In [56]:
torch.equal(a,b)

True

In [57]:
dropout.training

False

In [61]:
class MyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.lin1 = nn.Linear(in_features=input_dim, out_features=hidden_dim)
        self.lin2 = nn.Linear(in_features=hidden_dim, out_features=hidden_dim)
        self.lin3 = nn.Linear(in_features=hidden_dim, out_features=output_dim)
        self.relu = nn.ReLU()
        # dropout for pretraining and finetunning
        self.dropout_for_GD = nn.Dropout(p=0.5)
        # dropout for smc
        self.dropout = DropoutLayer(hidden_dim=hidden_dim)

    def forward(self, x):
        x = self.lin1(x)
        x = self.relu(x)
        x = self.lin2(x)
        x = self.relu(x)
        if self.training:
            x = self.dropout_for_GD(x)
        else:
            x = self.dropout(x)
        x = self.lin3(x)
        return x

    def update_dropout_mask(self, mask: np.ndarray):
        self.dropout.update(mask)

    def get_dropout_mask(self):
        return self.dropout.get()

In [112]:
model = MyModel()

In [63]:
print(model)

MyModel(
  (lin1): Linear(in_features=2, out_features=512, bias=True)
  (lin2): Linear(in_features=512, out_features=512, bias=True)
  (lin3): Linear(in_features=512, out_features=2, bias=True)
  (relu): ReLU()
  (dropout_for_GD): Dropout(p=0.5, inplace=False)
  (dropout): DropoutLayer()
)


# pretraining

In [67]:
loaded = np.load('./two_link_arm_pretraining_data.npz')
# print(np.array_equal(angles , loaded['x']))
# print(np.array_equal(positions , loaded['y']))
angles = loaded['x']
positions = loaded['y']

In [68]:
def train_test_split_reptile(x,y, test_size):
    x_train, x_test, y_train, y_test = train_test_split(np.swapaxes(x.reshape(n_tasks, n_episode, -1),0,1), np.swapaxes(y.reshape(n_tasks, n_episode, -1),0,1), test_size=test_ratio, shuffle = False)
    return x_train.reshape(-1,2), x_test.reshape(-1,2), y_train.reshape(-1,2),y_test.reshape(-1,2)

In [69]:
test_ratio = 0.1

In [70]:
x_train, x_test, y_train, y_test = train_test_split(angles, positions, test_size=test_ratio, random_state=666)

In [82]:
angles

array([[  5.20502714,   0.68284737],
       [  0.06279707, -10.12601557],
       [  4.72078754,   1.39050319],
       ...,
       [  2.79322114,  -7.7134945 ],
       [  0.20457976,   1.32375426],
       [ -0.14625527,   0.9901522 ]])

In [88]:
dataset = TensorDataset(torch.Tensor(angles), torch.Tensor(positions))
dataloader = DataLoader(dataset, batch_size=128)

In [90]:
for x, y in dataloader:
    print(x.shape)
    print(y.shape)
    break

torch.Size([128, 2])
torch.Size([128, 2])


In [113]:
def train_epochs(model, x_train, y_train, epochs=200, batch_size=128):
    dataset = TensorDataset(torch.Tensor(angles), torch.Tensor(positions))
    dataloader = DataLoader(dataset, batch_size=batch_size)
    optimizer = torch.optim.Adam(model.parameters(),lr=1e-4)
    criterion = nn.MSELoss()
    model.train()
    for epoch in range(epochs):
        train_loss = 0
        count = 0
        with tqdm(dataloader, unit='batch') as tepoch:
            for x, y in tepoch:
                tepoch.set_description(f"Epoch {epoch}")
                optimizer.zero_grad()
                y_pred = model(x)
                loss = criterion(y_pred, y)
                loss.backward()
                optimizer.step()

                train_loss += loss.item() * y_pred.size(0)
                count += y_pred.size(0)
                tepoch.set_postfix(loss=train_loss/count)
                # sleep(0.1)

In [114]:
train_epochs(model, angles, positions)

Epoch 3:  72%|███████▏  | 841/1172 [00:20<00:08, 41.16batch/s, loss=0.597]


KeyboardInterrupt: 